# Scala関数型デザイン&プログラミング
https://github.com/fpinscala/fpinscala/

## Chaper2
https://github.com/fpinscala/fpinscala/blob/master/exercises/src/main/scala/fpinscala/gettingstarted/GettingStarted.scala

In [5]:
def fib(x:Int):Int={
    @annotation.tailrec
    def loop(n:Int,prev:Int,cur:Int):Int={
        if (n==0) prev
        else loop(n-1,cur,prev+cur)
    }
    loop(x,0,1)
}

fib(4)

defined function fib
res4_1: Int = 3

In [7]:
// listにフィボナッチを格納する
def fib(x:Int):List[Int]={
    @annotation.tailrec
    def loop(n:Int,prev:Int,cur:Int,stc:List[Int]):List[Int]={
        if (n==0) stc.reverse
        else loop(n-1,cur,prev+cur,cur::stc)
    }
    loop(x,0,1,Nil)
}

fib(4)

defined function fib
res6_1: List[Int] = List(1, 1, 2, 3)

In [14]:
def isSorted[A](as:Array[A], gt:(A,A)=>Boolean):Boolean ={
        @annotation.tailrec
    def go(n:Int):Boolean={
        if (n>=(as.length-1)) true
        else if (gt(as(n),as(n+1))) false
        else go(n+1)
    }
    
    go(0)
}

isSorted(Array(1,2,3),(x:Int,y:Int)=>x>y)
isSorted(Array(1,3,2),(x:Int,y:Int)=>x>y)

defined function isSorted
res13_1: Boolean = true
res13_2: Boolean = false

In [31]:
// 部分適用
def partial1[A,B,C](a: A, f: (A,B) => C): B => C =
 (b:B)=>f(a,b)

val par=partial1(1,(a:Int,b:Int)=>a+b)

defined function partial1
par: Int => Int = <function1>

In [30]:
// カリー化
def curry[A,B,C](f: (A, B) => C): A => (B => C) =
(a:A)=>(b:B)=>f(a,b)

val cur=curry((a:Int,b:Int)=>a+b)
cur(1)

defined function curry
cur: Int => Int => Int = <function1>
res29_2: Int => Int = <function1>

In [27]:
def uncurry[A,B,C](f: A => B => C): (A, B) => C =
(a:A,b:B)=>f(a)(b)

val f=(a:Int)=>(b:Int)=>a+b
val unc=uncurry(f)
unc(1,2)

defined function uncurry
f: Int => Int => Int = <function1>
unc: (Int, Int) => Int = <function2>
res26_3: Int = 3

In [24]:
def compose[A,B,C](f:A=>B,g:B=>C):A=>C=
(a:A)=>g(f(a))

defined function compose

## Chapter3


In [9]:
sealed trait List[+A]
// 変位アノテーション(+A)とNothing型がすべての型の部分型であることを組み合わせより、NilをList[Int]など全ての型とみなすことができる
case object Nil extends List[Nothing]  // Noneはオブジェクトなので、何度呼び出してもメモリは消費されない
case class Cons[+A](head:A,tail:List[A]) extends List[A] // これはデフォルルトの :: に相当

object List {
    def sum(ints:List[Int]):Int=ints match {
        case Nil=>0
        case Cons(x,xs)=> x+sum(xs) 
    }
 
//     A*はA型の任意の数の引数を渡す事を表し、_*はSeqなどを分解し、複数の引数を渡す事を表す
    def apply[A](as:A*):List[A]={
        if(as.isEmpty) Nil
        else Cons(as.head,apply(as.tail:_*))
    }
}


defined trait List
defined object Nil
defined class Cons
defined object List

In [2]:
// ちなみにOption型の場合。
sealed abstract class Option[+A] extends Product with Serializable

final case class Some[+A](x:A) extends Option[A]
case object None extends Option[Nothing]


defined class Option
defined class Some
defined object None

http://www.scala-lang.org/api/2.11.7/#scala.collection.immutable.$colon$colon

### データ共有でパフォーマンスを改善する

In [2]:
//  パフォーマンス測定のためのコード
def time[R](block: => R): R = {  
    val t0 = System.nanoTime()
    val result = block    // call-by-name
    val t1 = System.nanoTime()
    println("Elapsed time: " + (t1 - t0) + "ns")
    result
}

defined function time

In [10]:
lazy val test1=(1 to 10000).map{_=>
    val list=(1 to 1000).toList
    1::list  
}

test1: collection.immutable.IndexedSeq[List[Int]] = <lazy>

In [11]:
lazy val test2=(1 to 10000).map{_=>
    val list=(1 to 1000).toList
    1+:list  
}

test2: collection.immutable.IndexedSeq[List[Int]] = <lazy>

In [12]:
time(test1)

Elapsed time: 18082303198ns


res11: collection.immutable.IndexedSeq[List[Int]] = Vector(
  List(
    1,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
...

In [13]:
time(test2)

Elapsed time: 25009607765ns


res12: collection.immutable.IndexedSeq[List[Int]] = Vector(
  List(
    1,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
...

##### 性能評価
18082303198ns  
25009607765ns

In [10]:
// ex 3.2

def tail[A](list:List[A]):List[A]= list match {
    case Cons(_,xs)=>xs
    case _=>Nil
}


def tail[A](list:scala.collection.immutable.List[A]):scala.collection.immutable.List[A]= list match {
    case _::xs=>xs
    case _=>scala.collection.immutable.Nil
}

tail(List(1,2,3))
tail(List(1))

defined function tail
defined function tail
res9_2: List[Int] = Cons(2,Cons(3,Nil))
res9_3: List[Int] = Nil

In [11]:
val b=scala.collection.immutable.List(1,2,3)
val scala.collection.immutable.List(x,_*)=b
// 先頭を取得する
x

// 後方を取得する
val scala.collection.immutable.List(y,ys@_*)=b
ys

b: collection.immutable.List[Int] = List(1, 2, 3)
x: Int = 1
res10_2: Int = 1
y: Int = 1
ys: Seq[Int] = List(2, 3)
res10_4: Seq[Int] = List(2, 3)

In [13]:
// ex 3.3
def setHead[A](x:A,list:List[A]):List[A]= list match {
    case Cons(_,xs)=>Cons(x,xs)
    case _=>Cons(x,Nil)
}

setHead(1,List(2,3))

defined function setHead
res12_1: List[Int] = Cons(1,Cons(3,Nil))

In [18]:
// ex 3.4

def drop[A](n:Int,list:List[A]):List[A]= list match {
    case Cons(_,xs) if (n>0)=>drop(n-1,xs)
    case xs=>xs
}

drop(2,List(1,2,3))
drop(0,List(1,2,3))
drop(4,List(1,2,3))

defined function drop
res17_1: List[Int] = Cons(3,Nil)
res17_2: List[Int] = Cons(1,Cons(2,Cons(3,Nil)))
res17_3: List[Int] = Nil

In [21]:
// ex 3.5 カリー化の特性を活かす
def dropWhile[A](list:List[A])(f:A=>Boolean):List[A]=
   list match{
       case Cons(x,xs) if f(x)=>dropWhile(xs)(f)
       case _=> list
   }


dropWhile(List(1,2,3,4))(x=>x<3)

defined function dropWhile
res20_1: List[Int] = Cons(3,Cons(4,Nil))

#### リスト再帰と高階関数の一般化

In [22]:
def foldRight[A,B](as:List[A],zero:B)(f:(A,B)=>B):B={
    as match {
        case Nil=> zero
        case Cons(x,xs)=> f(x,foldRight(xs,zero)(f))
    }
}


defined function foldRight

In [25]:
// この辺りの理論はMonad則と深い関わりがある
def sumInt(list:List[Int],zero:Int)=foldRight(list,zero)(_+_)
def sumStr(list:List[String],zero:String)=foldRight(list,zero)(_+_)

sumStr(List("a","b","c"),"")
sumInt(List(1,2,3),0)

defined function sum
defined function sumStr
res24_2: String = "abc"
res24_3: Int = 6

In [26]:
// val scalazVersion="7.1.0"
// classpath.add("org.scalaz" %% "scalaz-core" % scalazVersion)
// classpath.add("org.scalaz" %% "scalaz-effect" % scalazVersion)
// classpath.add("org.scalaz" %% "scalaz-typelevel" % scalazVersion)
// classpath.add("org.scalaz" %% "scalaz-scalacheck-binding" % scalazVersion )


Adding 1 artifact(s)
Adding 1 artifact(s)
Adding 1 artifact(s)
Adding 6 artifact(s)


scalazVersion: String = "7.1.0"

In [26]:
// import scalaz._
// import Scalaz._
// def sum[A](list:List[Monoid[A]],zero:Monoid[A])=foldRight(list,zero)((x,y)=>x |+| y)

: 

Scalaでは: で終わるメソッドはすべて右結合である  
つまり、`x::xs`は実際には`xs ::(x)`であり、:: メソッドは、データコンストラクタ`::(x,xs)`を呼び出してる

In [ ]:
@SerialVersionUID(509929039250432923L) // value computed by serialver for 2.11.2, annotation added in 2.11.4
final case class ::[B](override val head: B, private[scala] var tl: List[B]) extends List[B] {
  override def tail : List[B] = tl
  override def isEmpty: Boolean = false
}

In [ ]:
type ::[A] = scala.collection.immutable.::[A] 
val :: = scala.collection.immutable.::

In [ ]:
  /** Adds an element at the beginning of this list.
   *  @param x the element to prepend.
   *  @return  a list which contains `x` as first element and
   *           which continues with this list.
   *
   *  @usecase def ::(x: A): List[A]
   *    @inheritdoc
   *
   *    Example:
   *    {{{1 :: List(2, 3) = List(2, 3).::(1) = List(1, 2, 3)}}}
   */
  def ::[B >: A] (x: B): List[B] =
    new scala.collection.immutable.::(x, this)

In [1]:
// メソッドの::が呼び出される
1 ::List(1)

res0: List[Int] = List(1, 1)

In [ ]:
// 変数の::が呼ばれる
val a= List(1,2,3) match {
    case 1::2::3::Nil=> 1.0
}

: 